# Load Queries json

In [1]:
import json

data_filename = '../test_data/case_2.json'
with open(data_filename) as file:
    data = json.load(file)

In [2]:
queries = [query['query'] for query in data['queries']]
list_of_queries = [{'classes': data['classes'],"options": data['options'],'query':query} for query in queries]
query_strings = [json.dumps(d) for d in list_of_queries]


# Create Prompt

# Workflow

In [5]:
from src import utility_functions 
utility_functions.process_request(query_strings[0])

2024-04-30 06:32:49.193 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


{'error': 'Unexpected error occurred',
 'details': 'No secrets files found. Valid paths for a secrets.toml file are: /home/vscode/.streamlit/secrets.toml, /workspaces/classification-webapp-submission/development/.streamlit/secrets.toml'}

In [4]:
processed_queries  = list(map(utility_functions.process_request, query_strings))

In [3]:
!pip install httpx

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
classes = pd.DataFrame(data['classes'])
classes

,class_id,class_name,class_description
0,I1,Inform,The user is providing a piece of information
1,R1,Request,The user is requesting something
2,Q1,Question,The user is asking a question to gain information
3,C1,Command,The user is giving a command or directive
4,A1,Agreement,The user is agreeing with something that has b...
5,D1,Disagreement,The user is disagreeing with something that ha...
6,T1,Thanks,The user is expressing gratitude
7,Ap1,Apology,The user is apologizing for an action or state...
8,S1,Suggestion,The user is suggesting a course of action
9,F1,Feedback,The user is providing feedback or evaluation


In [6]:
import pandas as pd
classes = pd.DataFrame(data['classes'])
queries = pd.DataFrame(data['queries'])
multilabel = data['options']['multilabel']
output = queries.merge(pd.DataFrame(processed_queries),left_index=True, right_index=True)
output

,query,class_id,result,reasoning
0,"I'm going to Tottenham Court Road, can you tel...","[I1, R1]","[R1, Q1]",- Class ID 'R1': The user is requesting inform...
1,What time does the store close?,[Q1],[Q1],- Class ID 'Q1': The user asked a question see...
2,Please turn off the lights before you leave.,[C1],[C1],- Class ID 'C1': The user is giving a command ...
3,"Yes, I think that's a great idea!",[A1],[A1],- Class ID 'A1': The model identified an agree...
4,"Actually, I don't think that's how it works.",[D1],[D1],- Class ID 'D1': The model identified disagree...
5,Thank you so much for your help!,[T1],[T1],- Class ID 'T1': The user expressed gratitude ...
6,I'm really sorry for being late.,[Ap1],[Ap1],- Class ID 'Ap1': The model identified an apol...
7,Maybe you should try restarting your computer.,[S1],"[C1, S1]",- Class ID 'C1': The user is giving a command ...
8,I found the presentation very informative and ...,[F1],[F1],- Class ID 'F1': The user found the presentati...
9,"Is this the best method to solve the problem, ...","[Q1, S1]","[Q1, S1]",- Class ID 'Q1': The user is asking a question...


In [7]:
i=0
print('Query:',output.iloc[i].query)
for cs in output.iloc[i].class_id:
    print('Expected response:', cs, ' | Expected class name:',classes[classes.class_id==cs].class_name.values, '| Expected class description:', classes[classes.class_id==cs].class_description.values)
for cs in output.iloc[i].result:
    print('Predicted response:', cs, ' | Predicted class name:',classes[classes.class_id==cs].class_name.values, '| Predicted class description:', classes[classes.class_id==cs].class_description.values)
print('Predicted reason:',output.iloc[i].reasoning)


Query: I'm going to Tottenham Court Road, can you tell me how to get there?
Expected response: I1  | Expected class name: ['Inform'] | Expected class description: ['The user is providing a piece of information']
Expected response: R1  | Expected class name: ['Request'] | Expected class description: ['The user is requesting something']
Predicted response: R1  | Predicted class name: ['Request'] | Predicted class description: ['The user is requesting something']
Predicted response: Q1  | Predicted class name: ['Question'] | Predicted class description: ['The user is asking a question to gain information']
Predicted reason: - Class ID 'R1': The user is requesting information on how to get to Tottenham Court Road.
- Class ID 'Q1': The user is asking for directions or guidance on how to reach Tottenham Court Road.
